In [ ]:
import requests

# URL de ton API OCR
url = "https://19fe840519fb.ngrok-free.app/ocr"

# Chemin de l'image à envoyer
image_path = "/content/3.jpg"   # ou Desktop/3.jpeg si ton script est sur le bureau

# Ouvrir l’image
with open(image_path, "rb") as f:
    files = {"image": (image_path, f, "image/jpeg")}
    response = requests.post(url, files=files)

# Affichage du résultat
print("Status:", response.status_code)
print("Réponse OCR:", response.json())

with open(image_path, "rb") as f:
    data = {"image": (image_path, f, "/content/3.jpg")}
    resp = requests.post(url, files=data)

print(resp.json()["text"])


In [ ]:
import requests

url = "https://19fe840519fb.ngrok-free.app/ocr"
image_path = "/content/4.jpg"

with open(image_path, "rb") as f:
    response = requests.post(
        url,
        files={"image": (image_path, f, "image/jpeg")}
    )

data = response.json()

print("--- TEXT EXACTEMENT COMME SWAGGER ---")
print(data["text"])                 # même résultat que Swagger
print("--- JSON complet ---")
print(data)


Mon assistant :

corrige OCR

identifie les médicaments

génère leur fiche médicale (option B)

fait une recherche FAISS intelligente

génère une réponse médicale experte via Mistral

🔥 j'ai maintenant un assistant médical IA complet, gratuit, open-source, sans API, avec expert OCR + RAG enrichi.

                      (ancien code)
        OCR TEXT  ──►  LLM extraction  ──►  DRUGS CORRIGÉS
                                │
                                ▼
                          Matching dataset (FAISS)
                                │
            DRUG FOUND? ───────┼────► NO → ADD TO DATASET + REBUILD EMBEDDINGS
                                ▼
                            DRUG LIST FINAL
                                │
                                ▼
                  RAG (retrieved meds only) + llM
                                │
                                ▼
                    INTERACTION MÉDICALE INTELLIGENTE


In [ ]:
!pip uninstall -y transformers sentence-transformers


In [ ]:
!pip install transformers==4.30.0
!pip install sentence-transformers==2.2.2


In [ ]:
!pip install --upgrade huggingface_hub


In [ ]:
!pip install --upgrade sentence-transformers


In [ ]:
pip install httpx

In [ ]:
pip install openai


In [ ]:
import httpx
import json
from openai import OpenAI
import re

# --- Connexion API TokenFactory ---
esprit_api_key = "sk-e376096028c847389e18f6d1f650be93"

http_client = httpx.Client(verify=False)
client = OpenAI(
    api_key=esprit_api_key,
    base_url="https://tokenfactory.esprit.tn/api",
    http_client=http_client
)


# ------------------------------------------------------
# 🛡️ 1. JSON Cleaner : retire le markdown, répare virgules
# ------------------------------------------------------
def clean_json_output(text):
    text = text.strip()

    # Enlever éventuels ```json ... ```
    text = re.sub(r"```json", "", text, flags=re.IGNORECASE).strip()
    text = re.sub(r"```", "", text).strip()

    # Retirer trailing commas avant les }
    text = re.sub(r",\s*}", "}", text)
    text = re.sub(r",\s*]", "]", text)

    return text


# ------------------------------------------------------
# 🛡️ 2. VALIDATION DES CHAMPS
# ------------------------------------------------------
def validate_medical_card(card: dict, drug_name: str):
    """
    Vérifie que la carte contient bien tous les champs requis.
    Corrige les types si nécessaire.
    """

    template = {
        "drug": drug_name,
        "class": "",
        "indications": [],
        "mechanism": "",
        "dosage": "",
        "side_effects": [],
        "contraindications": [],
        "interactions": []
    }

    for key, default in template.items():

        if key not in card:
            card[key] = default
            continue

        # Assurer les types corrects
        if isinstance(default, list) and not isinstance(card[key], list):
            card[key] = [card[key]] if card[key] else []

        if isinstance(default, str) and not isinstance(card[key], str):
            card[key] = str(card[key])

    return card


# ------------------------------------------------------
# 🧠 3. FONCTION PRINCIPALE : ULTRA-ROBUSTE
# ------------------------------------------------------
def generate_medical_card(drug: str):
    """
    Génère une carte médicale fiable, nettoyée et validée.
    """

    prompt = f"""
Tu es un expert médical. Ta mission : générer UNE SEULE fiche médicale fiable
pour le médicament suivant : "{drug}".

RÈGLES STRICTES :
- Tu réponds EXCLUSIVEMENT en JSON valide.
- Pas de texte avant ou après le JSON.
- Remplis seulement ce que tu connais avec certitude.
- Si tu n'es pas sûr, mets une chaîne vide "" ou une liste vide [].

FORMAT EXACT À RESPECTER :
{{
  "drug": "{drug}",
  "class": "",
  "indications": [],
  "mechanism": "",
  "dosage": "",
  "side_effects": [],
  "contraindications": [],
  "interactions": []
}}
"""

    try:
        response = client.chat.completions.create(
            model="hosted_vllm/Llama-3.1-70B-Instruct",
            messages=[
                {"role": "system", "content": "Assistant médical strict, factuel, sans spéculation."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.15,   # 🔥 plus basse → moins d'hallucinations
            max_tokens=600
        )

        raw = response.choices[0].message.content.strip()

        # --- nettoyage JSON
        cleaned = clean_json_output(raw)

        try:
            parsed = json.loads(cleaned)
        except Exception:
            print("⚠ JSON invalide. Sortie brute :")
            print(raw)
            return {"drug": drug, "error": "Invalid JSON", "raw": raw}

        # Validation-type + complétion des champs
        validated = validate_medical_card(parsed, drug)

        return validated

    except Exception as e:
        print(f"⚠ Erreur génération carte médicale pour {drug}: {e}")
        return {"drug": drug, "error": str(e)}
card = generate_medical_card("Paracetamol")
print(json.dumps(card, indent=2, ensure_ascii=False))


In [ ]:
import re
import json
import httpx
from openai import OpenAI

# --- Connexion à l’API TokenFactory --- 
# Utilisation de httpx pour la connexion HTTP avec TokenFactory (Llama)
esprit_api_key = "sk-e376096028c847389e18f6d1f650be93"  # Remplacez par votre propre clé API ESPRIT

# --- Connexion à l’API ESPRIT ---
http_client = httpx.Client(verify=False)
client = OpenAI(
    api_key=esprit_api_key,
    base_url="https://tokenfactory.esprit.tn/api",
    http_client=http_client
)


def correct_medicine_name_with_llama(med_name, dosage, duration):
    """
    Appelle llama pour valider ou corriger un nom de médicament,
    en utilisant le dosage et la durée comme contexte pour plus de précision.
    """
    # Utilisation de f-strings pour inclure les détails contextuels
    prompt = f"""
    Vous êtes un expert en pharmacie et votre unique tâche est de valider le nom d'un médicament, en utilisant le dosage et la durée comme contexte pour garantir la précision.
    **Répondez uniquement avec le nom du médicament corrigé ou original, sans aucune explication ni ponctuation supplémentaire.**

    Informations de l'ordonnance:
    - Nom extrait (potentiellement erroné) : {med_name}
    - Posologie/Instructions : {dosage}
    - Durée : {duration}

    Correction :
    Si le nom est mal orthographié ou peu clair, proposez la correction la plus probable et pertinente dans ce contexte. Par exemple, si le nom est 'PaniD' et le dosage est '40mg before meals', la correction appropriée est 'Pan-D' (ou Pantoprazole). Si le nom est correct, laissez-le tel quel.

    Nom du médicament corrigé ou confirmé :
    """

    # Envoi de la requête via l'API ESPRIT
    try:
        response = client.chat.completions.create(
            model="hosted_vllm/Llama-3.1-70B-Instruct",
            messages=[ 
                {"role": "system", "content": "Tu es un assistant médical et administratif utile et concis."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.4,
            max_tokens=700,
            top_p=0.9
        )

        # Log de la réponse pour inspecter sa structure
        print("Réponse complète de l'API:", response)

        # Accéder à la réponse en extrayant les choix correctement
        content = response.choices[0].message.content  # Utiliser la bonne syntaxe pour accéder au contenu
        return content

    except Exception as e:
        print(f"⚠️ Erreur lors de la correction du médicament '{med_name}' : {e}")
        return {"drug": med_name, "error": str(e)}


def extract_and_correct_meds_with_llama(ocr_text):
    """
    Appelle llama pour extraire les médicaments, la posologie et la durée
    d'un texte OCR. Concentré sur l'extraction brute et le format JSON strict.
    """
    prompt = f"""
    Votre tâche est d'analyser le texte OCR d'une ordonnance médicale et d'**extraire tous les médicaments identifiables, leur posologie et leur durée**.
    Le texte peut contenir des erreurs d'orthographe ou de fusion. Identifiez le **nom de médicament le plus probable** à partir du contexte.
    **N'effectuez AUCUNE correction d'orthographe ni de validation pharmaceutique.** Indiquez seulement ce que vous trouvez.
    **Laissez le champ 'dosage' et 'duration' aussi bruts que possible** pour capturer toutes les informations contextuelles pertinentes.

    Vous DEVEZ sortir une réponse **stricte** au format JSON suivant :

    [
        {{"drug": "Nom du médicament extrait", "dosage": "Informations de posologie brutes (ex: 625mg x5day, 40mg before meals)", "duration": "Informations de durée brutes (ex: 5days, 1week)"}}
    ]

    TEXT OCR :
    {ocr_text}
    """

    # Envoi de la requête via l'API ESPRIT
    try:
        response = client.chat.completions.create(
            model="hosted_vllm/Llama-3.1-70B-Instruct",
            messages=[ 
                {"role": "system", "content": "Tu es un assistant médical et administratif utile et concis."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.4,
            max_tokens=700,
            top_p=0.9
        )

        # Log de la réponse pour inspecter sa structure
        print("Réponse complète de l'API:", response)

        # Accéder à la réponse en extrayant le contenu JSON depuis la réponse de l'API
        response_text = response.choices[0].message.content  # Utiliser la bonne syntaxe pour accéder au contenu
        print("Réponse textuelle extraite :", response_text)

        # Vérifier si la réponse est bien un JSON valide
        try:
            extracted_meds = json.loads(response_text)  # Convertir la chaîne JSON en un objet Python
        except json.JSONDecodeError as json_err:
            print(f"⚠️ Erreur lors du décodage JSON : {json_err}")
            return {"error": "Réponse API invalide"}

        return extracted_meds

    except Exception as e:
        print(f"⚠️ Erreur lors de l'extraction et correction des médicaments : {e}")
        return {"error": str(e)}


# --- EXEMPLE D'UTILISATION ---
ocr_text_input = "Rx Tab Auguentin 625mg x5day Enzoflarn 5days PaniD 40mg before meals Hexigel gum paste 1week"

# Extrait et corrige les médicaments extraits via Llama
extracted_meds = extract_and_correct_meds_with_llama(ocr_text_input)

# Vérifier si la réponse est bien un tableau de médicaments
if isinstance(extracted_meds, list):
    # Correction supplémentaire avec Llama si nécessaire
    corrected_meds = []
    for item in extracted_meds:
        corrected_name = correct_medicine_name_with_llama(item['drug'], item['dosage'], item['duration'])
        corrected_meds.append({
            "drug": corrected_name,
            "dosage": item['dosage'],
            "duration": item['duration']
        })

    # Afficher les résultats
    print("\n--- RÉSULTAT FINAL ---")
    print(json.dumps(corrected_meds, indent=2, ensure_ascii=False))
else:
    print(f"⚠️ Erreur : {extracted_meds.get('error', 'Erreur inconnue')}")


In [ ]:
import os
import re
import json
import random
import httpx
from openai import OpenAI

# ==============================
# 🔐 CONNEXION API ESPRIT
# ==============================
ESPRIT_API_KEY = os.getenv("ESPRIT_API_KEY", "sk-e376096028c847389e18f6d1f650be93").strip()
if not ESPRIT_API_KEY:
    raise RuntimeError("ESPRIT_API_KEY manquante. Définis la variable d'environnement ESPRIT_API_KEY.")

http_client = httpx.Client(verify=False, timeout=60.0)
client = OpenAI(
    api_key=ESPRIT_API_KEY,
    base_url="https://tokenfactory.esprit.tn/api",
    http_client=http_client
)

MODEL = "hosted_vllm/Llama-3.1-70B-Instruct"

# ==============================
# 🛡️ JSON SAFE PARSER
# ==============================
def safe_json_loads(llm_output: str):
    if not llm_output or not llm_output.strip():
        raise ValueError("Réponse LLM vide")

    cleaned = llm_output.strip()
    cleaned = re.sub(r"```json", "", cleaned, flags=re.IGNORECASE)
    cleaned = re.sub(r"```", "", cleaned)

    match = re.search(r"\{.*\}|\[.*\]", cleaned, re.DOTALL)
    if not match:
        raise ValueError(f"Aucun JSON trouvé.\n---RAW---\n{cleaned}\n----------")

    return json.loads(match.group(0))


# ==============================
# 🧠 (A) CLASSIFIEUR LLM : token -> MEDICAMENT/AUTRE
# ==============================
def classify_token_llm(token: str, ocr_text: str) -> str:
    """
    Retourne STRICTEMENT 'MEDICAMENT' ou 'AUTRE'
    (pas de JSON ici, pour être ultra robuste).
    """
    prompt = f"""
Vous êtes un classifieur strict.

TEXTE OCR :
{ocr_text}

TOKEN :
"{token}"

Question : ce token est-il un NOM DE MEDICAMENT ou un MOT ORDINAIRE ?

Répondez STRICTEMENT par un seul mot :
MEDICAMENT
ou
AUTRE
"""
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Tu réponds strictement au format demandé."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
        max_tokens=5,
        top_p=1.0
    )
    out = (resp.choices[0].message.content or "").strip().upper()
    if "MEDICAMENT" in out:
        return "MEDICAMENT"
    if "AUTRE" in out:
        return "AUTRE"
    # fallback : si le modèle dérape, on force une normalisation
    return "AUTRE"


# ==============================
# 🧩 (B) EXPLICATION LLM (déclarative) en JSON
# ==============================
def explain_token_llm(token: str, ocr_text: str, base_class: str) -> dict:
    """
    Explication XAI token-spécifique et comparative.
    """

    prompt = f"""
Vous êtes un expert en explicabilité IA pour le NLP médical.

Objectif :
Expliquer pourquoi le TOKEN analysé joue un rôle de NOM DE MEDICAMENT
plutôt qu'un rôle de mot ordinaire DANS CE TEXTE PRÉCIS.

IMPORTANT :
- L'explication doit être centrée sur le TOKEN, pas sur le texte global.
- Comparez explicitement le token à ses mots voisins.
- N'expliquez PAS pourquoi le texte est médical, mais pourquoi CE token est médical.

TEXTE OCR :
{ocr_text}

TOKEN ANALYSÉ :
"{token}"

DÉCISION DU MODÈLE :
{base_class}

FORMAT JSON STRICT :
{{
  "classification": "{base_class}",
  "token_role": "rôle fonctionnel du token dans la ligne",
  "local_evidence": [
    "indice local directement lié au token",
    "indice local lié à sa position ou relation"
  ],
  "contrast_with_neighbors": "pourquoi les mots voisins ne jouent pas le même rôle",
  "counterfactual_link": "quel élément local, s'il disparaît, fait changer la décision"
}}
"""

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Réponds uniquement en JSON strict."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=400,
        top_p=0.9
    )

    return safe_json_loads(response.choices[0].message.content or "")


# ==============================
# 🧪 (C) XAI CAUSALE : perturbations automatiques
# ==============================
def _tokenize_simple(text: str):
    # tokenisation simple, sans règles métier
    # on garde mots / nombres / unités collées comme dans OCR
    return re.findall(r"[A-Za-zÀ-ÿ]+|\d+(?:[.,]\d+)?|[^\s]", text)

def _join_tokens(tokens):
    # reconstruit un texte "proche" de l’original, sans casser trop
    return " ".join(tokens).replace(" ,", ",").replace(" .", ".").replace(" ;", ";").replace(" :", ":")

def generate_perturbations(token: str, ocr_text: str, n: int = 18, window: int = 6):
    """
    Génère des variantes du contexte autour du token (masquage/suppression locale),
    sans règles médicales codées.
    """
    toks = _tokenize_simple(ocr_text)
    # trouver une occurrence (approx) du token
    idxs = [i for i, t in enumerate(toks) if t.lower() == token.lower()]
    if not idxs:
        # si pas trouvé, on renvoie des perturbations globales (masquage aléatoire)
        idxs = [len(toks) // 2]
    center = idxs[0]

    variants = []
    for _ in range(n):
        vt = toks[:]  # copy
        left = max(0, center - window)
        right = min(len(vt), center + window + 1)

        # on choisit aléatoirement un sous-ensemble à masquer dans la fenêtre
        candidates = [i for i in range(left, right) if i != center]
        k = max(1, int(len(candidates) * random.uniform(0.25, 0.7)))
        chosen = random.sample(candidates, k=min(k, len(candidates)))

        # deux types de perturbation : MASK ou DROP (aléatoire)
        mode = random.choice(["MASK", "DROP"])
        for i in chosen:
            if mode == "MASK":
                vt[i] = "[MASK]"
            else:
                vt[i] = ""  # drop

        # on garde le token central
        vt[center] = toks[center]

        # nettoyage des tokens vides
        vt_clean = [t for t in vt if t != ""]
        variants.append(_join_tokens(vt_clean))

    # ajouter 2 variantes “hard” (sans contexte proche / contexte réduit)
    # (toujours sans règles médicales, juste manipulation de fenêtre)
    only_local = toks[max(0, center - window):min(len(toks), center + window + 1)]
    variants.append(_join_tokens(only_local))

    without_local = toks[:]
    for i in range(max(0, center - window), min(len(toks), center + window + 1)):
        if i != center:
            without_local[i] = "[MASK]"
    variants.append(_join_tokens(without_local))

    return variants


def causal_xai_via_perturbations(token: str, ocr_text: str, n: int = 18, window: int = 6) -> dict:
    """
    XAI forte :
    - calcule la stabilité de la classification sous perturbations
    - identifie des perturbations qui font basculer la décision (contrefactuels validés)
    """
    base = classify_token_llm(token, ocr_text)
    variants = generate_perturbations(token, ocr_text, n=n, window=window)

    results = []
    flips = []
    for v in variants:
        c = classify_token_llm(token, v)
        results.append({"variant_text": v, "classification": c})
        if c != base:
            flips.append({"variant_text": v, "classification": c})

    stability = 1.0 - (len(flips) / max(1, len(variants)))

    # pick a validated counterfactual (first flip)
    validated_cf = flips[0] if flips else None

    return {
        "base_classification": base,
        "stability": round(stability, 3),
        "num_variants": len(variants),
        "num_flips": len(flips),
        "validated_counterfactual": validated_cf,
        "all_variant_results": results  # tu peux enlever si trop long
    }


# ==============================
# 💊 Ton extraction (inchangée) + correction (inchangée)
# ==============================
def extract_meds_with_llama(ocr_text):

    prompt = f"""
Analysez le texte OCR d'une ordonnance.

Extraire tous les médicaments SANS corriger.
Retournez UNIQUEMENT un JSON strict.

FORMAT :
[
  {{
    "drug": "nom extrait",
    "dosage": "posologie brute",
    "duration": "durée brute"
  }}
]

TEXTE OCR :
{ocr_text}
"""

    response = client.chat.completions.create(
        model="hosted_vllm/Llama-3.1-70B-Instruct",
        messages=[
            {"role": "system", "content": "Tu es un assistant médical précis."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=700,
        top_p=0.9
    )

    return safe_json_loads(response.choices[0].message.content)




def correct_medicine_name_with_llama(med_name, dosage, duration):
    """
    Appelle llama pour valider ou corriger un nom de médicament,
    en utilisant le dosage et la durée comme contexte pour plus de précision.
    """
    # Utilisation de f-strings pour inclure les détails contextuels
    # Utilisation de f-strings pour inclure les détails contextuels
    prompt = f"""
    Vous êtes un expert en pharmacie, et votre tâche consiste à valider ou corriger le nom d'un médicament, en prenant en compte le contexte donné par le dosage et la durée.

    Veuillez procéder comme suit :
    1. Si le nom extrait semble erroné (par exemple, en raison d'erreurs typographiques ou d'orthographe), suggérez la correction la plus probable.
    2. Si le nom extrait est correct, laissez-le tel quel.
    3. Ne vous basez pas sur des règles fixes ou des corrections automatiques. Utilisez uniquement votre expertise pour choisir le nom correct en fonction du contexte.
    4. Ne donnez aucune explication supplémentaire. Limitez-vous au nom du médicament corrigé ou confirmé.
    
    Informations extraites de l'ordonnance :

    Nom extrait : {med_name}
    Dosage : {dosage}
    Durée : {duration}

    Nom final :
    """

    response = client.chat.completions.create(
        model="hosted_vllm/Llama-3.1-70B-Instruct",
        messages=[
            {"role": "system", "content": "Tu es un expert pharmaceutique."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=50,
        top_p=0.9
    )

    return response.choices[0].message.content.strip()




# ==============================
# 🚀 PIPELINE COMPLET + XAI VALIDÉE
# ==============================
def full_pipeline_with_validated_xai(ocr_text: str, n_perturb: int = 18, window: int = 6):
    extracted = extract_meds_with_llama(ocr_text)

    out = []
    for item in extracted:
        token = item.get("drug", "").strip()
        dosage = item.get("dosage", "")
        duration = item.get("duration", "")

        # 1) XAI causale (stabilité + contrefactuel validé)
        causal = causal_xai_via_perturbations(token, ocr_text, n=n_perturb, window=window)

        # 2) Explication déclarative (mais ancrée à la décision)
        expl = explain_token_llm(token, ocr_text, causal["base_classification"])

        # 3) Correction (si c'est un médicament)
        corrected = token
        if causal["base_classification"] == "MEDICAMENT":
            corrected = correct_medicine_name_with_llama(token, dosage, duration)

        out.append({
            "original_token": token,
            "classification": causal["base_classification"],
            "xai": {
                "explanation": expl,
                "causal_validation": {
                    "stability": causal["stability"],
                    "num_variants": causal["num_variants"],
                    "num_flips": causal["num_flips"],
                    "validated_counterfactual": causal["validated_counterfactual"]
                }
            },
            "corrected_drug": corrected,
            "dosage": dosage,
            "duration": duration
        })

    return out


# ==============================
# 🧪 RUN
# ==============================
if __name__ == "__main__":
    random.seed(42)

    ocr_text_input = (
        "Rx Tab Auguentin 625mg x5day "
        "Enzoflarn 5days "
        "PaniD 40mg before meals "
        "Hexigel gum paste 1week"
    )

    result = full_pipeline_with_validated_xai(ocr_text_input, n_perturb=14, window=5)
    print(json.dumps(result, indent=2, ensure_ascii=False))


Comment fonctionne ton XAI (en 4 étapes simples)
🔹 Étape 1 — Décision de base

Le LLM décide :

MEDICAMENT ou AUTRE
pour chaque token extrait de l’OCR.

🔹 Étape 2 — Explication locale (XAI déclarative)

Le modèle explique :

le rôle du token

les indices locaux (dosage, position, voisins)

pourquoi les mots autour ne sont pas des médicaments

ce qui ferait changer la décision (contre-exemple)

-> C’est une explication humaine, lisible

🔹 Étape 3 — Validation causale (XAI forte)

On modifie automatiquement le texte autour du token :

masquage

suppression locale

réduction du contexte

Puis on regarde :

si la décision reste stable

ou si elle bascule

-> Si la décision change, on a un contrefactuel validé

🔹 Étape 4 — Score de confiance implicite

À partir des perturbations, on calcule :

stability score (robustesse)

nombre de changements de décision

-> Plus c’est stable, plus la décision est fiable

In [ ]:
!pip install sentence-transformers
!pip install faiss-cpu


In [ ]:
!pip uninstall -y sentence-transformers huggingface-hub transformers


In [ ]:
!pip list | findstr transformer
!pip list | findstr hugging


In [ ]:
!pip install huggingface-hub==0.19.4
!pip install transformers==4.36.2
!pip install sentence-transformers==2.3.1


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
print("OK !")


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# ALWAYS CPU
embedder = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")

def compute_embeddings(text_list):
    embeddings = embedder.encode(text_list, convert_to_numpy=True, normalize_embeddings=True)
    return embeddings.astype("float32")


In [ ]:
import faiss

def create_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)  # cos similarity
    index.add(embeddings)
    return index


In [ ]:
def match_drug_embeddings(drug_name, df, index, embeddings, threshold=0.60):

    query_emb = compute_embeddings([drug_name])  # (1, 384)

    scores, indices = index.search(query_emb, k=1)
    score = scores[0][0]
    idx = indices[0][0]

    if score >= threshold:
        return df["canonical"].iloc[idx], score

    return None, score


In [ ]:
import pandas as pd
df = pd.read_csv(r"C:\Users\hp\Desktop\4ème\projet\medicaments_clean_for_ocr.csv")

# Compute dataset embeddings
dataset_embeddings = compute_embeddings(df["canonical"].tolist())

# Build index
index = create_faiss_index(dataset_embeddings)

# Query
drug = "ENZOFLan"
match, score = match_drug_embeddings(drug, df, index, dataset_embeddings)

print(match, score)


In [ ]:
import json
import pandas as pd

def add_drug_to_dataset(drug_name, medical_card, df, dataset_path):
    # Convert JSON → string
    med_card_str = json.dumps(medical_card, ensure_ascii=False)

    new_row = {
        "canonical": drug_name,
        "med_card": med_card_str
    }

    df = df.append(new_row, ignore_index=True)

    # Save to disk
    df.to_csv(dataset_path, index=False, encoding="utf-8")

    return df


In [ ]:
# Charger dataset
dataset_path = r"C:\Users\hp\Desktop\4ème\projet\medicaments_clean_for_ocr.csv"
df = pd.read_csv(dataset_path)

# Vérifier si la colonne existe sinon la créer
if "med_card" not in df.columns:
    df["med_card"] = ""  # colonne vide
    df.to_csv(dataset_path, index=False, encoding="utf-8")
    print("🆕 Colonne 'med_card' ajoutée au dataset.")
else:
    print("✔ Colonne 'med_card' déjà existante.")


In [ ]:
def process_drug(med, df, index, embeddings, dataset_path):

    # 1️⃣ Correction LLM
    corrected_name = correct_medicine_name_with_llama(
        med["drug"], med["dosage"], med["duration"]
    )
    print(f"\n🔧 Nom corrigé : {corrected_name}")

    # 2️⃣ Matching embedding sur NOM CORRIGÉ
    match, score = match_drug_embeddings(corrected_name, df, index, embeddings)

    # 3️⃣ Si médicament connu
    if match:
        print(f"✔ Match trouvé : {corrected_name} → {match} (score={score:.2f})")

        row = df[df["canonical"] == match].iloc[0]
        raw_card = row["med_card"]

        # ---- CAS : Carte déjà existante ----
        if isinstance(raw_card, str) and raw_card.strip() not in ["", "nan", "None"]:
            print(f"📄 Carte médicale trouvée pour {match}.")
            med_card = json.loads(raw_card)
            return match, med_card, df, index, embeddings

        # ---- CAS : Carte manquante → générer nouvelle carte ----
        print(f"⚠️ Carte médicale absente dans dataset pour {match}. Génération en cours…")
        med_card = generate_medical_card(match)

        # Mise à jour dataset
        df.loc[df["canonical"] == match, "med_card"] = json.dumps(med_card, ensure_ascii=False)
        df.to_csv(dataset_path, index=False, encoding="utf-8")

        return match, med_card, df, index, embeddings

    # 4️⃣ Aucun match → nouveau médicament
    print(f"❌ Aucun match pour {corrected_name} → génération carte médicale…")
    med_card = generate_medical_card(corrected_name)

    # Ajout dans dataset
    new_row = {
        "canonical": corrected_name,
        "med_card": json.dumps(med_card, ensure_ascii=False)
    }
    
    df = df.append(new_row, ignore_index=True)
    df.to_csv(dataset_path, index=False, encoding="utf-8")

    print(f"🆕 Médicament ajouté au dataset : {corrected_name}")

    # Rebuild embeddings & FAISS
    embeddings = compute_embeddings(df["canonical"].tolist())
    index = create_faiss_index(embeddings)

    return corrected_name, med_card, df, index, embeddings


In [ ]:
#######################################
# 🔥 TEST COMPLET DU PIPELINE 🔥
#######################################

# Emplacement réel de ton dataset
dataset_path = "C:/Users/hp/Desktop/4ème/projet/medicaments_clean_for_ocr.csv"

print("📌 Chargement du dataset...")
df = pd.read_csv(dataset_path)

# Ajouter colonne med_card si elle n'existe pas
if "med_card" not in df.columns:
    df["med_card"] = ""
    df.to_csv(dataset_path, index=False, encoding="utf-8")
    print("🆕 Colonne 'med_card' ajoutée.")

print("📌 Calcul des embeddings...")
embeddings = compute_embeddings(df["canonical"].tolist())

print("📌 Création de l'index FAISS...")
index = create_faiss_index(embeddings)


########################################
# 🔍 TEST OCR INPUT
########################################

ocr_text = """
Tab ENZOFLan 5mg x5day
PaniD 40mg before meals
Augmentin 1g x3day
"""

print("\n🧪 OCR fourni :")
print(ocr_text)


########################################
# 🧪 EXTRACTION BRUTE
########################################

extracted = extract_and_correct_meds_with_llama(ocr_text)

print("\n🔍 Médicaments extraits (brut OCR) :")
print(json.dumps(extracted, indent=2, ensure_ascii=False))


########################################
# 🔥 TRAITEMENT DE CHAQUE MÉDICAMENT
########################################

final_output = []

for med in extracted:
    drug_name, med_card, df, index, embeddings = process_drug(
        med, df, index, embeddings, dataset_path
    )

    final_output.append({
        "drug": drug_name,
        "dosage": med["dosage"],
        "duration": med["duration"],
        "card": med_card
    })


########################################
# 🎉 RESULTAT FINAL
########################################

print("\n\n🎉=== RÉSULTAT FINAL DU PIPELINE ===🎉")
print(json.dumps(final_output, indent=2, ensure_ascii=False))

print("\n📌 Vérification : dataset mis à jour → OK")
print("📌 Embeddings recalculés → OK")
print("📌 Index FAISS reconstruit → OK")

print("\n🎯 TEST TERMINÉ — TON PIPELINE FONCTIONNE ✔")


In [ ]:
def get_medical_context(drug_list, df):
    context_parts = []

    for drug in drug_list:
        # 🔹 match insensible à la casse
        mask = df["canonical"].astype(str).str.lower() == drug.lower()
        rows = df[mask]

        if rows.empty:
            print(f"⚠️ Aucun médoc trouvé dans le dataset pour : {drug}")
            continue

        row = rows.iloc[0]
        raw_card = row.get("med_card", "")

        # si pas de carte → on peut soit sauter, soit mettre un placeholder
        if pd.isna(raw_card) or raw_card in ["", "nan", None]:
            print(f"⚠️ med_card vide pour : {drug}")
            continue

        try:
            card = json.loads(raw_card)
        except Exception as e:
            print(f"⚠️ Erreur JSON pour med_card de {drug} : {e}")
            continue

        context_parts.append(json.dumps(card, ensure_ascii=False, indent=2))

    return "\n\n".join(context_parts)


In [ ]:
def build_rag_prompt(question, context):
    return f"""
Tu es un expert médical spécialisé dans les interactions médicamenteuses.

Contexte clinique provenant des cartes médicales extraites du dataset :

{context}

Règles :
- Utilise PRIORITAIREMENT ce contexte pour répondre.
- Tu peux compléter avec tes connaissances médicales internes si nécessaire.
- Répond toujours de manière claire, simple et exacte.
- Mentionne explicitement les interactions possibles.
- Donne éventuellement des recommandations pratiques.

Question :
{question}

Réponse :
"""



In [ ]:
def ask_medical_question(question, drug_list, df):
    context = get_medical_context(drug_list, df)
    prompt = build_rag_prompt(question, context)

    response = client.chat.completions.create(
        model="hosted_vllm/Llama-3.1-70B-Instruct",
        messages=[
            {"role": "system", "content": "Assistant médical fiable et prudent."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=500
    )

    return response.choices[0].message.content.strip()


In [ ]:
def ask_about_prescription(question, final_output, df):
    # Liste des médicaments reconnus après correction + matching
    drugs = [item["drug"] for item in final_output]

    print("\n📌 Médicaments concernés par la question :", drugs)

    answer = ask_medical_question(question, drugs, df)

    return answer


In [ ]:
question = "Est-ce que je peux prendre Augmentin avec Pantoprazole ?"

print("\n🧠 QUESTION :", question)

answer = ask_about_prescription(question, final_output, df)

print("\n💬 RÉPONSE RAG :\n")
print(answer)


In [ ]:
conversation_history = []  # mémoire du chat


def ask_medical_question_conversational(question, drug_list, df):
    # 1. Construire contexte médical
    context = get_medical_context(drug_list, df)

    # 2. Construire le message utilisateur pour ce tour
    user_message = f"""
Contexte des médicaments :
{context}

Question de l'utilisateur :
{question}
"""

    # 3. Construire la conversation complète
    messages = [{"role": "system", "content": "Assistant médical expert, prudent, clair et fiable."}]

    # Ajouter historique
    for entry in conversation_history:
        messages.append({"role": entry["role"], "content": entry["content"]})

    # Ajouter le nouveau message
    messages.append({"role": "user", "content": user_message})

    # 4. Appeler le modèle
    response = client.chat.completions.create(
        model="hosted_vllm/Llama-3.1-70B-Instruct",
        messages=messages,
        temperature=0.25,
        max_tokens=500
    )

    answer = response.choices[0].message.content.strip()

    # 5. Ajouter ce tour dans l'historique
    conversation_history.append({"role": "user", "content": question})
    conversation_history.append({"role": "assistant", "content": answer})

    return answer


In [ ]:
def chat_with_prescription(question, final_output, df):
    drugs = [item["drug"] for item in final_output]
    answer = ask_medical_question_conversational(question, drugs, df)
    return answer


In [ ]:
print("💬 Assistant Médical Intelligent — Mode Conversation")
print("Tape 'exit' pour quitter.\n")

while True:
    user_input = input("👤 Vous : ")

    if user_input.lower().strip() in ["exit", "quit"]:
        print("👋 Fin de la conversation.")
        break

    answer = chat_with_prescription(user_input, final_output, df)

    print("\n🧠 QUESTION :")
    print(user_input)

    print("\n🤖 RÉPONSE :")
    print(answer)
    print("\n" + "="*80 + "\n")


In [ ]:
import re
import json

def extract_json_block(text):
    """
    Extrait proprement un bloc JSON depuis une réponse LLM contenant du texte avant/après.
    """
    match = re.search(r"\{[\s\S]*\}", text)
    if match:
        json_text = match.group(0)
        try:
            return json.loads(json_text)
        except Exception as e:
            print("⚠️ JSON détecté mais impossible à parser :", e)
            return None
    return None


In [ ]:
def self_check_llm(context, question, answer):
    """
    Vérifie la réponse générée par le RAG :
      - erreurs médicales ?
      - interaction inventée ?
      - hallucination ?
      - réponse corrigée ?
    Retourne un texte LLM contenant du JSON.
    """

    prompt = f"""
Tu es un examinateur médical strict et objectif.

Analyse la réponse suivante et détecte :

1. Si une interaction médicamenteuse a été INVENTÉE.
2. Si la réponse contient une ERREUR MÉDICALE.
3. Si la réponse contient une HALLUCINATION.
4. Si la réponse peut présenter un RISQUE pour l’utilisateur.
5. Donne un score de certitude (0–100).
6. Si incorrect → propose une version corrigée.

Répond STRICTEMENT en JSON selon le modèle :

{{
  "invented_interaction": true/false,
  "medical_error": true/false,
  "hallucination": true/false,
  "risk": true/false,
  "certainty": "xx/100",
  "corrected_answer": "..."
}}

CONTEXT :
{context}

QUESTION :
{question}

ANSWER :
{answer}
"""

    resp = client.chat.completions.create(
        model="hosted_vllm/Llama-3.1-70B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=500
    )

    return resp.choices[0].message.content


In [ ]:
def ask_with_xai(question, final_output, df):
    """
    1. Construit le contexte RAG
    2. Génère une réponse (RAG)
    3. Vérifie la réponse via XAI (self-check)
    4. Applique un safety-layer si risque détecté
    """

    # 1) médicaments concernés
    drugs = [item["drug"] for item in final_output]

    # 2) Contexte RAG
    context = ""
    context_cards = {}
    for drug in drugs:
        card = get_medical_context([drug], df)
        context_cards[drug] = card
        context += f"\n\n[{drug.upper()}]\n{card}"

    # 3) réponse brute
    raw_answer = ask_medical_question_conversational(question, drugs, df)

    # 4) attribution
    retrieval = explain_retrieval(question, context_cards)

    # 5) Self-check
    report_raw = self_check_llm(context, question, raw_answer)
    quality = extract_json_block(report_raw)

    # -------------------------------
    # 🌙 6) Sélection de la réponse finale
    # -------------------------------

    safety_note = ""
    final_answer = raw_answer  # valeur par défaut

    if quality is None:
        final_answer = raw_answer
        safety_note = "⚠️ Le self-check n’a pas pu être analysé."
    
    else:
        invented = quality.get("invented_interaction", False)
        med_error = quality.get("medical_error", False)
        risk = quality.get("risk", False)
        corrected = quality.get("corrected_answer", "").strip()

        # 6A – utiliser la réponse corrigée si erreur détectée
        if invented or med_error:
            final_answer = corrected if corrected else raw_answer
            safety_note = "⚠️ Réponse initiale corrigée (erreur détectée par XAI)."

        # 6B – 🌋 SAFETY-LAYER : risques détectés  
        if risk:
            final_answer = (
                "⚠️ Cette question concerne une interaction médicale potentiellement risquée. "
                "Je ne peux pas fournir de réponse définitive. "
                "Veuillez demander l'avis d’un médecin ou d’un pharmacien."
            )
            safety_note = "🛑 Réponse bloquée (risque détecté par le modèle)."

        # 6C – réponse validée
        if not invented and not med_error and not risk:
            safety_note = "✔ Réponse validée par le self-check."

    return {
        "question": question,
        "answer_raw": raw_answer,
        "answer_final": final_answer,
        "quality_report": quality,
        "retrieval": retrieval,
        "safety_note": safety_note
    }


In [ ]:
def evaluate_system(questions, final_output, df):
    """
    Exécute N questions et retourne un DataFrame avec :
    - réponse brute
    - réponse finale
    - erreurs détectées
    - métriques XAI
    """

    logs = []

    for q in questions:
        print(f"\n🔍 QUESTION : {q}")
        result = ask_with_xai(q, final_output, df)

        report = result["quality_report"]
        invented = med_error = halluc = risk = None
        certainty = None

        if report:
            invented = report["invented_interaction"]
            med_error = report["medical_error"]
            halluc = report["hallucination"]
            risk = report["risk"]
            certainty = report["certainty"]

        logs.append({
            "question": q,
            "raw_answer": result["answer_raw"],
            "final_answer": result["answer_final"],
            "invented_interaction": invented,
            "medical_error": med_error,
            "hallucination": halluc,
            "risk": risk,
            "certainty": certainty,
            "safety_note": result["safety_note"]
        })

    return pd.DataFrame(logs)


In [ ]:
questions_test = [
    "Que faire si j’ai oublié une dose d’Augmentin ?",
    "Puis-je prendre deux Pantoprazole si j’ai oublié la dose d’hier ?",
    "Combien de jours faut-il pour que l’Augmentin fasse effet ?",
]

df_eval = evaluate_system(questions_test, final_output, df)

df_eval


In [ ]:
def compute_global_metrics(df_eval):
    """
    df_eval doit contenir les colonnes :
    - invented_interaction
    - medical_error
    - hallucination
    - risk
    - certainty (xx/100)
    """

    n = len(df_eval)

    def pct(col):
        return round(df_eval[col].fillna(False).mean() * 100, 2)

    taux_interaction = pct("invented_interaction")
    taux_erreur = pct("medical_error")
    taux_hallu = pct("hallucination")
    taux_risk = pct("risk")

    # Certitude moyenne
    certitudes = []
    for c in df_eval["certainty"]:
        try:
            certitudes.append(float(c.replace("/100", "")))
        except:
            pass

    certitude_moy = round(sum(certitudes) / len(certitudes), 2) if certitudes else None

    # SCORE GLOBAL DE SÉCURITÉ (0 = dangereux, 100 = excellent)
    score_securite = max(0, 100 - (taux_interaction * 1.5 + taux_erreur * 2 + taux_hallu * 1.2 + taux_risk * 0.5))
    score_securite = round(score_securite, 2)

    metrics = {
        "Nombre de questions": n,
        "Taux interactions inventées (%)": taux_interaction,
        "Taux erreurs médicales (%)": taux_erreur,
        "Taux hallucinations (%)": taux_hallu,
        "Taux réponses à risque (%)": taux_risk,
        "Certitude moyenne (%)": certitude_moy,
        "Score global de sécurité /100": score_securite
    }

    return metrics


In [ ]:
metrics = compute_global_metrics(df_eval)

print("\n📊 MÉTRIQUES GLOBALES DU SYSTÈME\n")
for k, v in metrics.items():
    print(f"- {k}: {v}")


In [ ]:
def rule_based_medical_logic(question):
    q = question.lower()

    # 1️⃣ OUBLI DE DOSE
    if "oubli" in q or "oublié" in q:
        return (
            "Si vous avez oublié une dose, prenez-la dès que possible sauf s’il est "
            "presque l’heure de la suivante. Ne doublez jamais la dose."
        )

    # 2️⃣ DOUBLE DOSE
    if ("deux" in q or "double" in q) and "dose" in q:
        return (
            "Il n’est pas recommandé de doubler une dose. Attendez la prochaine prise prévue."
        )

    # 3️⃣ DÉLAI D'EFFET
    if "combien de jours" in q or "agit" in q or "effet" in q:
        return (
            "L’Augmentin et la plupart des antibiotiques commencent à agir en 24 à 48 heures. "
            "Si les symptômes persistent ou s'aggravent, consultez un médecin."
        )

    # 4️⃣ ALCOOL
    if "alcool" in q:
        return (
            "Évitez l’alcool pendant un traitement, car il peut augmenter les effets secondaires "
            "digestifs ou altérer la tolérance du médicament."
        )

    # 5️⃣ REPAS
    if "repas" in q or "manger" in q:
        return (
            "Certains médicaments doivent être pris à jeun (comme le Pantoprazole), "
            "et d’autres avec un repas (comme l’Augmentin)."
        )

    return None


In [ ]:
def ask_with_xai(question, final_output, df):
    """
    Pipeline complet :
    1. Rule-based
    2. RAG
    3. Self-check
    4. Safety-layer
    """

    # -----------------------------
    # 0️⃣ RULE-BASED (anti-hallucination)
    # -----------------------------
    rule_answer = rule_based_medical_logic(question)
    if rule_answer is not None:
        return {
            "question": question,
            "answer_raw": rule_answer,
            "answer_final": rule_answer,
            "quality_report": None,
            "retrieval": [],
            "safety_note": "✔ Réponse sécurisée via module rule-based (pas de LLM utilisé)."
        }

    # -----------------------------
    # 1️⃣ Construire le contexte RAG
    # -----------------------------
    drugs = [item["drug"] for item in final_output]
    context = ""
    context_cards = {}

    for drug in drugs:
        card_text = get_medical_context([drug], df)
        context_cards[drug] = card_text
        context += f"\n[{drug.upper()}]\n{card_text}\n"

    # -----------------------------
    # 2️⃣ Réponse brute via RAG LLM
    # -----------------------------
    raw_answer = ask_medical_question_conversational(question, drugs, df)

    # FILTRE : si le modèle dit "info absente du contexte"
    if "absente du contexte" in raw_answer.lower():
        return {
            "question": question,
            "answer_raw": raw_answer,
            "answer_final": raw_answer,
            "quality_report": None,
            "retrieval": [],
            "safety_note": "ℹ️ L’information n’est pas présente dans le contexte RAG."
        }

    # Récupération pour attribution
    retrieval = explain_retrieval(question, context_cards)

    # -----------------------------
    # 3️⃣ SELF-CHECK LLM
    # -----------------------------
    report_raw = self_check_llm(context, question, raw_answer)
    quality = extract_json_block(report_raw)

    final_answer = raw_answer
    safety_note = ""

    # Aucune analyse possible → montrer RAG brut (rare)
    if quality is None:
        return {
            "question": question,
            "answer_raw": raw_answer,
            "answer_final": raw_answer,
            "quality_report": None,
            "retrieval": retrieval,
            "safety_note": "⚠️ Self-check illisible. Réponse brute affichée."
        }

    invented = quality.get("invented_interaction", False)
    med_error = quality.get("medical_error", False)
    risk = quality.get("risk", False)
    corrected = quality.get("corrected_answer", "").strip()

    # -----------------------------
    # 4️⃣ Correction automatique
    # -----------------------------
    if invented or med_error:
        final_answer = corrected if corrected else raw_answer
        safety_note = "⚠️ Réponse corrigée (erreur détectée par self-check)."

    # -----------------------------
    # 5️⃣ SAFETY-LAYER (niveau médical)
    # -----------------------------
    if risk:
        final_answer = (
            "⚠️ Cette question implique un risque médical potentiel. "
            "Je ne peux pas fournir de réponse définitive. "
            "Veuillez consulter un médecin ou un pharmacien."
        )
        safety_note = "🛑 Réponse bloquée (risque détecté par le self-check)."

    # -----------------------------
    # 🔚 6️⃣ Retour final
    # -----------------------------
    return {
        "question": question,
        "answer_raw": raw_answer,
        "answer_final": final_answer,
        "quality_report": quality,
        "retrieval": retrieval,
        "safety_note": safety_note
    }


In [ ]:
##############################################
# 🔥 TESTS COMPLETS DU SYSTÈME RAG + RULE + XAI
##############################################

test_questions = [
    "Que faire si j’ai oublié une dose d’Augmentin ?",
    "Puis-je prendre deux Pantoprazole si j’ai oublié la dose d’hier ?",
    "Combien de jours faut-il pour que l’Augmentin fasse effet ?",
    "Est-ce que je peux prendre Augmentin avec Pantoprazole ?",
    "Puis-je boire de l’alcool pendant un traitement Augmentin ?",
    "Est-ce que le Pantoprazole doit être pris avant ou après le repas ?",
]

results = []

print("\n==============================")
print("🧪 LANCEMENT DES TESTS")
print("==============================")

for q in test_questions:
    print(f"\n🔍 QUESTION : {q}")

    r = ask_with_xai(q, final_output, df)

    print("\n🤖 RÉPONSE FINALE :")
    print(r["answer_final"])

    if r["quality_report"] is not None:
        print("\n📊 SELF-CHECK :")
        print(r["quality_report"])

    print("\n🛡️ NOTE SÉCURITÉ :", r["safety_note"])

    # Sauvegarder résultats pour métriques globales
    results.append({
        "question": q,
        "raw": r["answer_raw"],
        "final": r["answer_final"],
        "quality": r["quality_report"],
        "risk": r["quality_report"]["risk"] if r["quality_report"] else False,
        "invented": r["quality_report"]["invented_interaction"] if r["quality_report"] else False,
        "hallucination": r["quality_report"]["hallucination"] if r["quality_report"] else False,
        "certainty": int(r["quality_report"]["certainty"].replace("/100","")) if r["quality_report"] else 100
    })


##############################################
# 📊 MÉTRIQUES GLOBALES
##############################################

print("\n\n==============================")
print("📊 MÉTRIQUES GLOBALES DU SYSTÈME")
print("==============================")

df_eval = pd.DataFrame(results)

num = len(df_eval)

risk_rate = 100 * df_eval["risk"].sum() / num
invented_rate = 100 * df_eval["invented"].sum() / num
halluc_rate = 100 * df_eval["hallucination"].sum() / num
certainty_avg = df_eval["certainty"].mean()

# Score global pondéré
global_score = (
    (100 - risk_rate) * 0.5 +
    (100 - invented_rate) * 0.3 +
    certainty_avg * 0.2
)

print(df_eval[["question", "final", "certainty", "risk", "invented", "hallucination"]])

print("\n📌 NOMBRE DE QUESTIONS :", num)
print(f"📌 Taux réponses à risque (%) : {risk_rate:.1f}")
print(f"📌 Taux interactions inventées (%) : {invented_rate:.1f}")
print(f"📌 Taux hallucinations (%) : {halluc_rate:.1f}")
print(f"📌 Certitude moyenne (%) : {certainty_avg:.1f}")
print(f"\n🏆 SCORE GLOBAL DE SÉCURITÉ /100 : {global_score:.1f}")


In [ ]:
pip install pdfplumber

In [ ]:
pip install langchain


pdf reader et pdf chat

                        PDF
                        │
                        ▼
                        Extraction du contenu (texte / structure)
                        │
                        ▼
                        Indexation (chunks + embeddings)
                        │
                        ▼
                        RAG
                        │
                        ▼
                        LLM
                        │
                        ▼
                        Réponses conversationnelles sur le document


In [ ]:
# --- Import robuste pour LangChain Splitter ---
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    print("✔ Import depuis langchain.text_splitter")
except ImportError:
    try:
        from langchain_text_splitters import RecursiveCharacterTextSplitter
        print("✔ Import depuis langchain_text_splitters")
    except ImportError:
        try:
            from langchain_core.text_splitter import RecursiveCharacterTextSplitter
            print("✔ Import depuis langchain_core.text_splitter")
        except ImportError:
            raise ImportError(
                "❌ Impossible d'importer RecursiveCharacterTextSplitter.\n"
                "Installe une version récente :\n"
                "pip install -U langchain langchain-community langchain-core"
            )


In [ ]:
!pip install -U langchain langchain-core langchain-community



In [ ]:
import langchain
print(langchain.__version__)


In [ ]:
import pdfplumber
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
import httpx
from openai import OpenAI
import asyncio
import os


In [ ]:
def extract_text_from_pdfs(pdf_paths):
    """
    pdf_paths : liste de chemins PDF
    """
    text = ""
    for pdf in pdf_paths:
        with pdfplumber.open(pdf) as doc:
            for page in doc.pages:
                content = page.extract_text()
                if content:
                    text += content + "\n"
    return text.strip()


In [ ]:
def create_vector_store(text):
    """
    Transforme le texte PDF en chunks + embeddings + FAISS retriever
    """

    chunks = RecursiveCharacterTextSplitter(
        chunk_size=8000,
        chunk_overlap=1000
    ).split_text(text)

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    return FAISS.from_texts(chunks, embeddings)


In [ ]:
def build_medical_prompt(question, context):
    template = """
Tu es un assistant médical intelligent.

RÔLE :
- Si le document contient des résultats médicaux : explique leur signification.
- Si c’est une ordonnance : identifie les médicaments et leur usage.
- Si c’est un rapport : résume les conclusions.
- Si le texte est administratif : explique les démarches.
- Si le texte est incomplet : donne la meilleure interprétation possible.

⚠️ RÈGLES :
- Pas de spéculation.
- Pas d’invention médicale.
- Utilise uniquement le CONTEXTE fourni.
- Si l’information manque → dire "Donnée absente du document".

📘 CONTEXTE :
{context}

❓ QUESTION :
{question}

💬 RÉPONSE :
"""

    prompt = ChatPromptTemplate.from_template(template)
    return prompt.format(context=context, question=question)


In [ ]:
def call_esprit_llm(prompt, esprit_api_key):
    http_client = httpx.Client(verify=False)

    client = OpenAI(
        api_key=esprit_api_key,
        base_url="https://tokenfactory.esprit.tn/api",
        http_client=http_client
    )

    response = client.chat.completions.create(
        model="hosted_vllm/Llama-3.1-70B-Instruct",
        messages=[
            {"role": "system", "content": "Tu es un assistant médical et administratif utile et concis."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
        max_tokens=700,
        top_p=0.9
    )

    return response.choices[0].message.content


In [ ]:
def analyse_pdf_chat(question, pdf_paths, esprit_api_key):
    # Extraction du texte PDF
    text = extract_text_from_pdfs(pdf_paths)
    if not text:
        return "❌ Aucun texte détecté dans les PDF."

    # Création du store FAISS
    vector_store = create_vector_store(text)

    retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    docs = retriever.invoke(question)

    if not docs:
        return "❌ Aucun passage pertinent trouvé."

    # Context
    context = "\n\n".join([d.page_content for d in docs])

    # Prompt intelligent
    prompt = build_medical_prompt(question, context)

    # Appel API
    return call_esprit_llm(prompt, esprit_api_key)


In [ ]:
esprit_api_key = "sk-e376096028c847389e18f6d1f650be93"

question = "Explique les résultats de cette ordonnance."
pdf_paths = ["C:\\Users\\hp\\Desktop\\4ème\\projet\\zahrouni maya.pdf"]

response = analyse_pdf_chat(question, pdf_paths, esprit_api_key)
print("\n🧠 RÉPONSE :\n")
print(response)



In [ ]:
import pdfplumber
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# --- Extraire texte depuis un PDF ---
def extract_text_from_pdf(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

# --- Construire le vector store ---
def create_vector_store_from_text(text):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=300
    )
    
    chunks = splitter.split_text(text)
    
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    
    vector_store = FAISS.from_texts(chunks, embedding_model)
    return vector_store


In [ ]:
pdf_path = r"C:\Users\hp\Desktop\4ème\projet\zahrouni maya.pdf"

# 1. Extraction texte
pdf_text = extract_text_from_pdf(pdf_path)

# 2. Construction vector store
vector_store = create_vector_store_from_text(pdf_text)

print("✔ vector_store créé avec succès")


In [ ]:
def xai_self_check(question, answer, context, esprit_api_key):
    prompt = f"""
Tu es un expert en évaluation RAG et sécurité médicale.
Analyse la réponse du modèle STRICTEMENT selon le contexte fourni.

Donne un JSON STRICT avec les champs suivants :

{{
 "faithfulness": 0,
 "relevance": 0,
 "completeness": 0,
 "risk": false,
 "errors": [],
 "final_score": 0
}}

Définitions :

- "faithfulness" = 1 si la réponse correspond STRICTEMENT au contexte PDF, sinon 0.
- "relevance" = score 0 à 100 de pertinence par rapport à la question.
- "completeness" = 1 si la réponse inclut toutes les infos du contexte nécessaires.
- "risk" = true si la réponse peut être dangereuse, fausse, ou spéculative.
- "errors" = liste détaillée des hallucinations ou contradictions.
- "final_score" = score pondéré :  
      50% faithfulness + 30% relevance + 20% completeness.

📘 CONTEXTE PDF :
{context}

❓ QUESTION :
{question}

💬 RÉPONSE DU MODÈLE :
{answer}

Répond uniquement en JSON.
    """

    http_client = httpx.Client(verify=False)
    client = OpenAI(
        api_key=esprit_api_key,
        base_url="https://tokenfactory.esprit.tn/api",
        http_client=http_client
    )

    response = client.chat.completions.create(
        model="hosted_vllm/Llama-3.1-70B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=500
    )

    raw = response.choices[0].message.content.strip()

    # nettoyage fallback
    try:
        return json.loads(raw)
    except:
        print("⚠️ JSON invalide retourné, affichage brut :")
        print(raw)
        return {
            "faithfulness": 0,
            "relevance": 0,
            "completeness": 0,
            "risk": True,
            "errors": ["Invalid JSON from LLM", raw],
            "final_score": 0
        }


In [ ]:
def analyse_pdf_chat(question, pdf_text, vector_store, esprit_api_key):
    # Retrieve context from vector store
    try:
        docs = vector_store.similarity_search(question, k=3)
        context = "\n\n".join([d.page_content for d in docs])
    except:
        context = pdf_text

    # Build prompt
    prompt = f"""
Tu es un assistant médical intelligent.
Analyse ces informations :

📘 CONTEXTE :
{context}

❓ QUESTION :
{question}

Réponse claire et concise :
"""

    # Call ESPRIT API
    http_client = httpx.Client(verify=False)
    client = OpenAI(
        api_key=esprit_api_key,
        base_url="https://tokenfactory.esprit.tn/api",
        http_client=http_client
    )

    response = client.chat.completions.create(
        model="hosted_vllm/Llama-3.1-70B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
        max_tokens=400
    )

    answer = response.choices[0].message.content.strip()

    return answer, context


In [ ]:
def test_and_evaluate(question, pdf_text, vector_store, esprit_api_key):
    print("\n==============================")
    print(f"🧠 QUESTION : {question}")
    print("==============================")

    # --- 1. Get RAG answer
    answer, context = analyse_pdf_chat(question, pdf_text, vector_store, esprit_api_key)

    print("\n🤖 RÉPONSE DU MODÈLE :\n")
    print(answer)

    # --- 2. Evaluate answer with XAI
    evaluation = xai_self_check(question, answer, context, esprit_api_key)

    print("\n📊 RAPPORT XAI :")
    print(evaluation)

    return {
        "question": question,
        "answer": answer,
        "context": context,
        "evaluation": evaluation
    }


In [ ]:
esprit_api_key = "sk-e376096028c847389e18f6d1f650be93"

question = "Explique les résultats de cette ordonnance."

result = test_and_evaluate(question, pdf_text, vector_store, esprit_api_key)

print("\n🎉 RÉSUMÉ FINAL :")
print(result)


In [ ]:
import pdfplumber
import json
import httpx
from openai import OpenAI

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

import pandas as pd
import numpy as np


In [ ]:
def extract_text_from_pdf(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            t = page.extract_text()
            if t:
                text += t + "\n"
    return text.strip()


In [ ]:
def create_vector_store(text):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=150
    )
    chunks = splitter.split_text(text)

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    vs = FAISS.from_texts(chunks, embeddings)
    return vs


In [ ]:
def build_medical_analysis_prompt(question, context):

    return f"""
Tu es un assistant médical ultra-fiable spécialisé dans l’analyse de documents biologiques.

Tu dois répondre STRICTEMENT à partir du texte fourni.
Aucune connaissance extérieure n’est autorisée.

────────────────────────────────────
📄 CONTEXTE (Données du document)
────────────────────────────────────
{context}

────────────────────────────────────
⚠️ RÈGLES ANTI-HALLUCINATION
────────────────────────────────────
1. Ne JAMAIS interpréter sans comparer aux valeurs normales présentes dans le document.
2. Ne JAMAIS écrire des phrases comme : “peut indiquer”, “probablement”, “infection”, “inflammation”.
3. Ne PAS inventer de diagnostic ou de maladie.
4. Si une information n'est pas dans le document → écrire : “Non indiqué dans le document.”
5. Réponse courte, factuelle et maximum 8 lignes.
6. Ne pas réécrire tout le document.

────────────────────────────────────
❓ QUESTION UTILISATEUR
{question}

────────────────────────────────────
🧪 RÉPONSE FACTUELLE (basée EXCLUSIVEMENT sur le document) :
"""


In [ ]:
def ask_llm(prompt):
    response = client.chat.completions.create(
        model="hosted_vllm/Llama-3.1-70B-Instruct",
        messages=[
            {"role": "system", "content": "Assistant médical strictement factuel."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.0
    )
    return response.choices[0].message.content


In [ ]:
def evaluate_answer(context, answer):
    errors = []

    # FAITHFULNESS = vérifier si les phrases ne sortent pas du contexte
    faithfulness = 100 if answer.lower() in context.lower() or any(w in context.lower() for w in answer.lower().split()) else 0

    # RELEVANCE = question pertinente par rapport au document
    relevance = 100 if len(set(answer.split()) & set(context.split())) > 5 else 60

    # COMPLETENESS = la réponse couvre-t-elle les valeurs évoquées
    completeness = 100 if "mm" in answer or "g/l" in answer or "%" in answer else 0

    # RISK = contient-il des mots médicaux dangereux ?
    risky_words = ["infection", "inflammation", "maladie", "insuffisance", "risque", "grave"]
    risk = any(w in answer.lower() for w in risky_words)

    return {
        "faithfulness": faithfulness,
        "relevance": relevance,
        "completeness": completeness,
        "risk": risk,
        "errors": errors,
        "final_score": (faithfulness*0.4 + relevance*0.3 + completeness*0.3)
    }


In [ ]:
def analyse_pdf_chat(question, pdf_path):
    text = extract_text_from_pdf(pdf_path)
    vector_store = create_vector_store(text)

    docs = vector_store.similarity_search(question, k=4)
    context = "\n\n".join([d.page_content for d in docs])

    prompt = build_medical_analysis_prompt(question, context)
    answer = ask_llm(prompt)

    eval_report = evaluate_answer(context, answer)

    return {
        "question": question,
        "answer": answer,
        "context_used": context,
        "evaluation": eval_report
    }


In [ ]:
pdf_path = r"C:\Users\hp\Desktop\4ème\projet\zahrouni maya.pdf"
question = "Explique les résultats de cette ordonnance."

result = analyse_pdf_chat(question, pdf_path)

print("\n🧠 RÉPONSE :\n", result["answer"])
print("\n📊 RAPPORT XAI :\n", result["evaluation"])


In [ ]:
questions = [
    "Explique les globules rouges.",
    "Y a-t-il des anomalies dans la NFS ?",
    "La créatinine est-elle normale ?",
    "Résumé de toute l’analyse."
]

for q in questions:
    print("\n==============================")
    print("🧠 QUESTION :", q)
    print("==============================")

    result = analyse_pdf_chat(q, pdf_path)

    print("\n🤖 RÉPONSE :\n", result["answer"])
    print("\n📊 XAI :\n", result["evaluation"])
